# Determining the longest diagonal of masks from MaskRCNN-Translated output on C3S-only dataset

## Dependencies

Import MaskRCNN code from GitHub

In [ ]:
!git clone 'https://github.com/Rene-Michel99/Mask-RCNN-TF2.8.git'

In [ ]:
!mv ./Mask-RCNN-TF2.8/* ./
!rm -rf ./Mask-RCNN-TF2.8
!rm -rf assets
!rm -rf tests

Install requirements

In [ ]:
!pip install -r ./requirements.txt

Import MaskRCNN libraries

In [ ]:
from mrcnn.Dataset import load_images_dataset
from mrcnn.model import MaskRCNN
from mrcnn.Utils import visualize
from mrcnn.Configs import Config

Common libraries

In [ ]:
!pip install roboflow
!pip install scikit-learn

In [ ]:
import math
import numpy as np
import cv2 as cv
import os
from datetime import datetime

## Dataset

Download from Roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="sDHrdDWd5USqSBR0E7on")
project = rf.workspace("medusas").project("alitas-tcc_h")
dataset = project.version(11).download("coco")

In [ ]:
DATA_SET_NAME = dataset.name.replace(" ", "-")
DATA_SET_LOCATION = dataset.location
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"

Dataset preprocessing (apply bilateral filter)

In [ ]:
def preprocess_coco_dataset(dataset_path, quiet=False):
    BILATERAL_PARAMETERS = (2, 52, 84)

    if not quiet:
        print(f"Preprocessing data on: {dataset_path}")

    img_filenames = os.listdir(dataset_path)
    img_filenames.remove("_annotations.coco.json")

    for filename in img_filenames:
        if not quiet:
            print(f"Applying Bilateral Filter on {filename}")
        try:
            img = cv.imread(os.path.join(dataset_path, filename))
            img = cv.bilateralFilter(img, *BILATERAL_PARAMETERS)
            cv.imwrite(os.path.join(dataset_path, filename))
        except:
            pass

    if not quiet:
        print(f"Data preprocessed successfully on {dataset_path}")


datasets_paths = [os.path.join(DATA_SET_LOCATION, 'train'),
                 os.path.join(DATA_SET_LOCATION, 'test'),
                 os.path.join(DATA_SET_LOCATION, 'valid'),]

for dataset in datasets_paths:
    preprocess_coco_dataset(dataset)

Prepare dataset for inference

In [ ]:
TEST_SET_PATH = os.path.join(DATA_SET_LOCATION, "test")

dataset_test = load_images_dataset(os.path.join(TEST_SET_PATH, ANNOTATIONS_FILE_NAME), TEST_SET_PATH, "test")

number_of_classes = dataset_test.count_classes()

print('Test: ', len(dataset_test.image_ids))
print('Class names: ', dataset_test.class_names)
print("Classes: ", number_of_classes)

## Network

### Configure network for inference

Set output directory

In [ ]:
OUTPUT_DIR_PATH = os.path.join(
    "output",
    datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
)

os.makedirs(OUTPUT_DIR_PATH, exist_ok=True)

Configure model for inference

In [ ]:
cfg_kwargs = {
    "name": "alitas_tcc_h",
    "num_classes": number_of_classes,
    "interpolation_method": "bilinear",
    "images_per_gpu": 1
}

config = Config(**cfg_kwargs)
model = MaskRCNN(mode="inference", config=config)

In [ ]:
config.display()

Load pre-trained weights

In [ ]:
PRE_TRAINED_WEIGHTS_PATH = "pretrained_weights/mask_rcnn_alitas_tcc_h_0003.h5"

model.load_weights(filepath=PRE_TRAINED_WEIGHTS_PATH, by_name=True)

### Predict

perform inference on test dataset with created predictor

In [ ]:
def inference_on_dataset(dataset, quiet=False):

    predictions = {}

    for img_id in dataset_test.image_ids:
        img_path = dataset_test.image_info[img_id].get("path")

        img = cv.imread(img_path)
        img = img[:,:,::-1]
        
        if not quiet:
            print(f"running inference on: {img_path}")
        
        detection = model.detect([img], verbose=0)[0]
        predictions[img_path] = detection
    
    return predictions 
    
predictions = inference_on_dataset(dataset_test)

## Measuring

Define measuring function to calculate biggest diagonal using PCA

In [ ]:
from sklearn.decomposition import PCA

def calc_longest_diagonal_pca(contour):

    contour = np.squeeze(contour)

    # handle single-point contours
    if (len(contour.shape) == 1):
        return tuple(contour), tuple(contour), 0

    pca = PCA(n_components=1)
    pca.fit(contour)

    principal_component = pca.components_[0]
    contour_pca = np.dot(contour, principal_component)

    start_index = np.argmin(contour_pca)
    end_index = np.argmax(contour_pca)

    start, end = contour[start_index], contour[end_index]
    start, end = tuple(start), tuple(end)
    length = math.dist(start, end)

    return start, end, length

Calculate longest diagonal with defined function

In [ ]:
def get_all_diagonals(predictions):
    diagonals = {}

    for (img_file_path, prediction) in predictions.items():

        masks = prediction["masks"]

        debug_weird_imgs = [] # debug
        diagonals_of_img_masks = []

        masks = np.where(masks == 1, 255, 0)
        masks = masks.astype(np.uint8)

        for mask in np.moveaxis(masks, -1, 0):
            contours,_ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
            
            # get the contour with the most points
            contour = max(contours, key=lambda x: x.shape[0])
            diagonal = calc_longest_diagonal_pca(contour)
            diagonals_of_img_masks.append(diagonal)

        diagonals[img_file_path] = diagonals_of_img_masks
        
    return diagonals

diagonals = get_all_diagonals(predictions)

Display measured diagonals

In [ ]:
IMAGES_SCALE_PX2NM = 0.8315

def draw_diagonal_on_img(img, diag_start, diag_end, color=(200,200,200), thickness=2):
    cv.line(img, diag_start, diag_end, color, thickness)

for (img_file_path, prediction) in predictions.items():

    img = cv.imread(img_file_path)
    masks = prediction["masks"]
    diagonals_img = diagonals[img_file_path]

    for i in range(masks.shape[-1]):
        start, end, length_pixels = diagonals_img[i]

        img_copy = np.array(img)
        draw_diagonal_on_img(img_copy, start, end)
        print("diagonal length (px): ", length_pixels)
        length_in_nanometers = length_pixels * IMAGES_SCALE_PX2NM
        print("diagonal length (nm): ", length_in_nanometers)

        display_kwargs = {
            "image": img_copy,
            "boxes": prediction["rois"],
            "masks": prediction["masks"],
            "class_ids": prediction["class_ids"],
            "class_names": ["" for _ in range(len(prediction["class_ids"]))],
            # "scores": prediction["scores"],
        }

        visualize.display_instances(**display_kwargs)